# GEE SYSI (cloudless)

Python implemntation of code by Sorenson et al., 2021, with changes added from Dvorakova et al. (2023) S2_cloudless and NBR2 threshold of 0.05



In [1]:
#import necessary packages
import ee
from IPython.display import Image
import folium
import geehydro

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWgavdeqwr_DCrk2DXiPFYVV5gA69Ow5t7Zq9-BRMStSU-23seQq08xsELY

Successfully saved authorization token.


In [3]:
#S2_cloudless parameters
#for full description: https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

AOI = ee.Geometry.Point(11.50, 44.98)
POLYGON = ee.Geometry.Polygon([[11.004490043487136, 44.44422303449173], [11.811985160674636,44.44422303449173], [11.811985160674636,45.20008726989075], [11.004490043487136,45.20008726989075], [11.004490043487136,44.44422303449173]])
START_DATE = '2017-01-01'
END_DATE = '2019-12-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 30
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [4]:
#create a S2_SR collection
S2collection = (ee.ImageCollection("COPERNICUS/S2_SR")
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        .filterBounds(POLYGON)
        .filterDate(START_DATE, END_DATE)
        .filter(ee.Filter.calendarRange(3,6,"month"))
        .select (['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'SCL' ])
        .sort("system:time_start"))


#create a S2_cloud_probability collection
S2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(POLYGON)
        .filterDate(START_DATE, END_DATE)
        .filter(ee.Filter.calendarRange(3,6,"month")))


#join the two collections above by system_index (acquisiiton date/time)
joined = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': S2collection,
        'secondary': S2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


#Image(url=S2first.getThumbUrl({'min': 0, 'max': 2000, 'bands':['B4','B3','B2'],'dimensions': 512}))

In [9]:
#A function to define cloud mask component

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))



#define cloud shadow mask component 
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


#function to add cloud and cloud shadow mask
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

S2joined = joined.map(add_cld_shdw_mask)
S2joined_cloudfree = S2joined.map(apply_cld_shdw_mask)
print(S2joined.toList(1).getInfo())

[{'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 600000, 0, -10, 5000040]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 600000, 0, -10, 5000040]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 600000, 0, -10, 5000040]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32632', 'crs_transform': [20, 0, 600000, 0, -20, 5000040]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32632', 'crs_transform': [20, 0, 600000, 0, -20, 500004

In [10]:
#Function to calculate and add an NDVI band
def addNDVI(image):
    return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('ndvi'))

# Function to calculate and add an NDWI band
def addNDWI(image):
    return image.addBands(image.normalizedDifference(['B3', 'B8']).rename('ndwi'))

#Function to calculate and add an NDI7 band   //ASLO KNOWN AS NBRI https://giscrack.com/list-of-spectral-indices-for-sentinel-and-landsat/ 
def addNDI7(image):
    return image.addBands(image.normalizedDifference(['B8', 'B12']).rename('ndi7'))

#Function to calculate and add an NBR2 band
def addNBR(image):
    return image.addBands(image.normalizedDifference(['B11', 'B12']).rename('nbr'))


#need to add a built up index.


S2joined = S2joined.map(addNDVI).map(addNDWI).map(addNDI7).map(addNBR)
S2joined_cloudfree = S2joined_cloudfree.map(addNDVI).map(addNDWI).map(addNDI7).map(addNBR)

NDVImax = S2joined.select('ndvi')
NDVImax = NDVImax.max()

NDVImax_cloudfree = S2joined_cloudfree.select('ndvi')
NDVImax_cloudfree = NDVImax_cloudfree.max()


In [19]:
#Calculate NDVI values
def ndviThres(image):
    #threshold: if NDVI less or equal to 0.3 => 1 else 0
    thres = image.select(['ndvi']).lte(0.3).rename('ndvi_thres')
    return image.addBands(thres)

def ndwiThres(image):
    # if nbr less or equal to 0.1 => 1 else 0
    thres = image.select(['ndwi']).lte(0.5).rename('ndwi_thres')
    return image.addBands(thres)

def NDI7Thres(image):
    # if nbr less or equal to 0.1 => 1 else 0
    thres = image.select(['ndi7']).lte(0).rename('ndi7_thres')
    return image.addBands(thres)

def nbrThres(image):
    # if nbr less or equal to 0.1 => 1 else 0
    thres = image.select(['nbr']).lte(0.05).rename('nbr_thres')
    return image.addBands(thres)


#Add the four band ratios
S2joined = S2joined.map(ndviThres).map(ndwiThres).map(NDI7Thres).map(nbrThres)
S2joined_cloudfree = S2joined_cloudfree.map(ndviThres).map(ndwiThres).map(NDI7Thres).map(nbrThres)

#mask ndvi values
def mask1(image):
    return image.updateMask(image.select('ndvi_thres'))

def mask2(image):
    return image.updateMask(image.select('ndwi_thres'))

def mask3(image):
    return image.updateMask(image.select('ndi7_thres'))

def mask4(image):
    return image.updateMask(image.select('nbr_thres'))


#print(S2joined.toList(1).getInfo())

#Threshold everything
collection = S2joined.map(mask1).map(mask2).map(mask3).map(mask4)
collection_noclouds = S2joined_cloudfree.map(mask1).map(mask2).map(mask3).map(mask4)
#print(collection_noclouds.toList(1).getInfo())


#compute the median for each pixel
median_noclouds = collection_noclouds.median()
median = collection.median()

#option to mask to ensure that there is vegetation at some point in the pixel
#by default this does not run
#mask by max ndvi

mask4 = NDVImax.gte(0.2)
mask4_cloudfree = NDVImax_cloudfree.gte(0.2)
median = median.mask(mask4)
median_noclouds = median_noclouds.mask(mask4)

#convert to float
median_noclouds = median_noclouds.float()
median = median.float()

#focal median filter the results to smooth out pixel to pixel variation
#useful for larger scale trend analysis vs detailed landscape analysis
median_noclouds = median_noclouds.focal_median(10).selfMask()
median = median.focal_median(10).selfMask()


In [25]:
#let's plot everything in folium

clouds = median.select('clouds').selfMask()
shadows = median.select('shadows').selfMask()
dark_pixels = median.select('dark_pixels').selfMask()
probability = median.select('probability')
cloudmask = median.select('cloudmask').selfMask()
cloud_transform = median.select('cloud_transform')


#initialise map
m = folium.Map(location= [44.96931692192336, 11.508487846221511], zoom_start=9)
m.setOptions('HYBRID') # To see GE map underneath
#map.addLayer(median.select("nbr").clip(Ferrara), {'min': -1, 'max': 1}, 'NBR')
#map.addLayer(median.select("ndi7").clip(Ferrara), {'min': -1, 'max': 1}, 'NDI7')
#map.addLayer(median.select("ndwi").clip(Ferrara), {'min': -1, 'max': 1}, 'NDWI')
#map.addLayer(median.select("ndvi").clip(Ferrara), {'min': -1, 'max': 1}, 'NDVI')
m.addLayer(median_noclouds.clip(POLYGON), {'min': 0, 'max': 2000, 'bands':['B4','B3','B2']}, 'RGB_noclouds')
m.addLayer(median.clip(POLYGON), {'min': 0, 'max': 2000, 'bands':['B4','B3','B2']}, 'RGB_withclouds')

#m.add_ee_layer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},'S2 image', True, 1, 9)
#m.add_ee_layer(s2_sr_median, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},'S2 cloud-free mosaic', True, 1, 9)

#m.addLayer(probability, {'min': 0, 'max': 100},'probability (cloud)', False, 9)
#m.addLayer(clouds, {'palette': 'e056fd'}, 'clouds', False, 9)
#m.addLayer(cloud_transform, {'min': 0, 'max': 1, 'palette': ['white', 'black']}, 'cloud_transform', False, 9)
#m.addLayer(dark_pixels, {'palette': 'orange'}, 'dark_pixels', False, 9)
#m.addLayer(shadows, {'palette': 'yellow'}, 'shadows', False, 9)
#m.addLayer(cloudmask, {'palette': 'orange'}, 'cloudmask', False, 9)


folium.LayerControl(collapsed=False).add_to(m) 


m


In [ ]:
## add bare soil data...
## extract spectra...
##train RF classifier
##classify
